In [1]:
# Write snapshots to temp file for reading xyz2graph

from xyz2graph import MolGraph, to_networkx_graph, to_plotly_figure
from plotly.offline import init_notebook_mode, iplot
import networkx as nx
import os, shutil
from datetime import datetime
from collections import defaultdict

from pyvis.network import Network
import pydot
from networkx.drawing.nx_pydot import *

nt = Network()
nt = Network(height="1500px", width="100%")

Tgraph = nx.DiGraph()  # Graph of transitions

#os.mkdir("Graphs")

start = datetime.now()

P = 0
NAME = ""

for D in ['Co']:#, 'Co', 'Cr', 'Cu', 'Fe', 'Hf', 'Hg', 'Ir', 'La', 'Mo', 'Nb', 'Ni', 'Os', 'Pd', 'Pt', 'Re', 'Rh', 'Sc', 'Ta', 'Tc', 'Ti', 'V', 'W', 'Y', 'Zn', 'Zr']:

    directory = D
    NAME += D

    Cgraph = nx.DiGraph()  # Graph of transitions
    print("Analyzing " + directory + "...")

    for x in os.listdir(directory):
        if x.endswith(".xyz") and not os.stat(directory + "/" + x).st_size == 0:
            # if x.endswith(".xyz"):
            name = x.strip(".xyz")

            print(name + "...")

            G = {}  # Dictionary of molecular graph for the current snapshot
            M = {}  # Dictionary of Unique confomer graphs
            Freq = {}
            Eng = {}

            RelEng = defaultdict(list)
            KRelEng = defaultdict(list)
            MinKRelEng = defaultdict(list)
            Refmtd = defaultdict(list)
            Barrier = defaultdict(list)
            ConfTrack = defaultdict(list)
            interaction = defaultdict(list)
            FP = defaultdict(list)

            # Initiate the Plotly notebook mode
            init_notebook_mode(connected=True)

            # i=2 #line number counter
            # k=1 #Snapshots counter
            # m = 1 # Unique conformer counter

            num_lines = sum(1 for line in open(directory + "/" + name + ".xyz"))

            if os.path.exists("configurations/" + name):

                shutil.rmtree("configurations/" + name)
                os.mkdir("configurations/" + name)

            f = open(directory + "/" + name + ".xyz")

            lines = f.readlines()
            natoms = int(lines[0])
            # RefEng = float(lines[1].split()[0])
            # print(RefEng)
            # Refmtd = lines[1].split()[1].strip('!').strip('mtd')
            # print(Refmtd)

            ## Isomorphism check - Database

            Freq = defaultdict(int)

            i = 2  # line number counter
            k = 0  # Snapshots counter
            m = 0  # Unique conformer counter

            # Tgraph = nx.DiGraph() # Graph of transitions

            start = datetime.now()
            while i + natoms < num_lines + 1:

                with open("snap.xyz", "w") as f:
                    for j in range(i - 2, i + natoms):
                        f.write(lines[j])

                ## read molecular graphs
                # Create the MolGraph object
                snap = MolGraph()
                # Read the data from the .xyz file
                snap.read_xyz("snap.xyz")
                k += 1
                # if k > 0 :
                #    Barrier[k] = (Eng[k]-Eng[k-1])*625

                G[name + str(k)] = (to_networkx_graph(snap)).to_undirected()
                Eng[name + str(k)] = float(lines[i - 1].split()[0])
                Refmtd[name + str(k)] = lines[i - 1].split()[1].strip("!").strip("mtd")

                check = 0

                for l in range(1, m + 1):
                    if nx.is_isomorphic(G[name + str(k)], M[name + str(l)]):
                        check = 1
                        ##Add total relative energy here:
                        ConfTrack[name + str(k)] = l
                        RelEng[(name + str(l), name + str(k))] = (
                            Eng[name + str(k)] - Eng[name + str(1)]
                        ) * 625
                        KRelEng[name + str(l)].append(
                            (Eng[name + str(k)] - Eng[name + str(1)]) * 625
                        )
                        Freq[name + str(l)] += 1
                        Q1 = str(ConfTrack[name + str(k - 1)])
                        if Q1 != "[]":
                            # print(Q1)
                            # Tgraph.add_edge(name+Q1 ,name+str(l))
                            if not (name + Q1, name + str(l)) in Cgraph.edges:
                                Cgraph.add_edge(name + Q1, name + str(l))

                if check == 0:
                    m += 1
                    M[name + str(m)] = G[name + str(k)]
                    ConfTrack[name + str(k)] = m
                    RelEng[(name + str(m), name + str(k))] = (
                        Eng[name + str(k)] - Eng[name + str(1)]
                    ) * 625
                    KRelEng[name + str(m)].append(
                        (Eng[name + str(k)] - Eng[name + str(1)]) * 625
                    )
                    Freq[name + str(m)] += 1
                    # Tgraph.add_node(i, REng=min(RelEng[i]), Frequency=Freq[i])
                    Q2 = str(ConfTrack[name + str(k - 1)])
                    # print(Q2)
                    if Q2 != "[]":
                        # Tgraph.add_edge(name+Q2 ,name+str(m))
                        Cgraph.add_edge(name + Q2, name + str(m))
                        
                    #print(m,k)
                    # shutil.move('snap.xyz','configurations/'+name+'/'+name+'-'+str(m)+'.xyz')

                # print(Eng[k])
                i += natoms + 2

            print(name + " " + str(m))
            # print(datetime.now()-start)

            P += m
            # Tgraph.add_edge(name ,name+'1')
            Cgraph.add_edge(name, name + "1")
            # Tgraph.add_edge(directory , name)
            Cgraph.add_edge(directory, name)

    Tgraph.add_edge("core", directory)
    nx.drawing.nx_pydot.write_dot(Cgraph, "Graphs/" + directory + ".gv")

nx.drawing.nx_pydot.write_dot(Tgraph, NAME + ".gv")

os.remove("snap.xyz")

print("Total number of explored unique configurations= " + str(P))
print(datetime.now() - start)

ModuleNotFoundError: No module named 'xyz2graph'

In [2]:
%pip install xyz2graph

  Could not find a version that satisfies the requirement xyz2graph (from versions: )
No matching distribution found for xyz2graph
You are using pip version 9.0.3, however version 22.0.3 is available.
You should consider upgrading via the 'python -m pip install --upgrade pip' command.
